In [ ]:
#group 12

In [ ]:
import requests
from bs4 import BeautifulSoup
import codecs
import csv
import os
import re
from tqdm.notebook import tqdm
import time
import pandas as pd
#!pip install newspaper3k
from newspaper import Article
import nltk
nltk.download('punkt')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# scrape data from the "Politics and Conflict" section of the Wikinews 

In [24]:
url = 'https://en.wikinews.org/wiki/Category:Politics_and_conflicts'

In [25]:
# initilize the home page
response = requests.get(url)
contents = response.text
soup = BeautifulSoup(contents, 'html.parser')
current_page = soup.find(id = "mw-pages")
# regexp of "next page url"
reg = r"<a href=\"\/w\/index\.php\?title=Category:Politics_and_conflicts&amp;(pagefrom=[^#]*#mw-pages)\" title=\"Category:Politics and conflicts\">next page</a>"
next_page = re.findall(reg, str(current_page))

In [ ]:
# write csv header
with codecs.open('/content/drive/MyDrive/Colab Notebooks/Final/news.csv','w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['title','scrape_at','url', 'pub_date', 'content','category','keyword','summary'])

# collect news
while next_page:
  soup = BeautifulSoup(contents, 'html.parser')
  current_page = soup.find(id = "mw-pages")
  news_row = []
  # find news title start with 'MNOPRSTUVW' in current page
  filtered = current_page.findAll(name='a',attrs={"href":re.compile(r'^(\/wiki\/[\s\S]*)')}) #'^(/wiki/[MNOPRSTUVW])')})
  if filtered:
      for i in filtered:
        # news title and url
        title = i.get_text()
        title_url = 'https://en.wikinews.org'+i['href']
        news_page = BeautifulSoup(requests.get(title_url).text, 'html.parser').find(class_="mw-parser-output")
        # publish date
        pub_date = [i.get('title') for i in news_page.findAll(name='span',attrs={"id":re.compile(r'publishDate')})]
        pub_date = ''.join(pub_date)
        # scrape time
        scrape_at = time.strftime("%Y.%m.%d %H:%M:%S", time.localtime())
        # category word
        cat_word = BeautifulSoup(requests.get(title_url).text, 'html.parser').find(class_="catlinks")
        category =[i.get_text() for i in cat_word.findAll(name='a',attrs={"href":re.compile(r'(\/wiki\/Category:)\b(?!Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\s\S]*[^"]')})]
        category = ','.join(category)
        # use newspaper3k and nlp to generate info about news
        article = Article(title_url)
        article.download()
        article.html
        article.parse()
        article.nlp()
        # content
        content = article.text
        # keyword
        key_word = article.keywords
        # summary
        summary = article.summary
        news_row.append([title, scrape_at, title_url, pub_date, content, category, key_word, summary])
      with codecs.open('/content/drive/MyDrive/Colab Notebooks/Final/news.csv','a+', encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in news_row:
            writer.writerow(data)
  next_page = re.findall(reg, str(current_page))
  if next_page:
    response = requests.get("https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&" + next_page[0])
    contents = response.text
  #time.sleep(0.01)

In [5]:
# read csv
CSV_PATH = '/content/drive/MyDrive/Colab Notebooks/Final/news.csv'
df = pd.read_csv(CSV_PATH)
#df=df.drop(labels=range(0,10),axis=0)
#df=df.reset_index(drop=True)

In [7]:
# add type
df['type']='political'
# add domain
df['domain']='wikinews.org'
# add domain
df['author']='wikinews'

In [23]:
df.head(5)

,title,scrape_at,url,pub_date,content,category,keyword,summary,type,domain,author
0,Ma Ying-jeou becomes President of Republic of ...,2021.06.01 16:53:08,https://en.wikinews.org/wiki/Ma_Ying-jeou_beco...,2008-05-20,"Tuesday, May 20, 2008\n\nToday, Ma Ying-jeou w...","Translated news,Published,Archived,Politics an...","['yuan', 'president', 'ma', 'winwin', 'hopes',...","Tuesday, May 20, 2008Today, Ma Ying-jeou was s...",political,wikinews.org,wikinews
1,Ma Ying-jeou wins 2008 Taiwan presidential ele...,2021.06.01 16:53:08,https://en.wikinews.org/wiki/Ma_Ying-jeou_wins...,2008-03-22,"Saturday, March 22, 2008\n\n\n\n\n\nMa Ying-je...","Asia,Politics and conflicts,Taiwan,Published,A...","['taiwan', 'ma', 'wins', 'president', 'ties', ...",The results of the election has been confirmed...,political,wikinews.org,wikinews
2,Macedonia says compromise with Greece over nam...,2021.06.01 16:53:08,https://en.wikinews.org/wiki/Macedonia_says_co...,2009-10-09,"Friday, October 9, 2009\n\nMacedonian Presiden...","North Macedonia,Published,Politics and conflic...","['yugoslaviasources', 'greece', 'dispute', 'ma...","Friday, October 9, 2009Macedonian President Gj...",political,wikinews.org,wikinews
3,Macedonian police kill four militants in mount...,2021.06.01 16:53:09,https://en.wikinews.org/wiki/Macedonian_police...,2007-11-07,"Wednesday, November 7, 2007\n\nAt least four m...","North Macedonia,Published,Archived,Europe,Crim...","['village', 'told', 'kosovo', 'šar', 'spokesma...","Locals from the village of Brodec, a Šar Mount...",political,wikinews.org,wikinews
4,Macedonian policeman killed in attack near Kosovo,2021.06.01 16:53:09,https://en.wikinews.org/wiki/Macedonian_police...,2007-10-25,"Thursday, October 25, 2007\n\nA Macedonian pol...","North Macedonia,Published,Archived,Serbia,Euro...","['village', 'kosovo', 'wounded', 'scene', 'yes...","Thursday, October 25, 2007A Macedonian police ...",political,wikinews.org,wikinews


In [21]:
# save to csv
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Final/wiki.csv')